In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import MBartForConditionalGeneration, MBartTokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from rouge_score import rouge_scorer

c:\Anaconda\envs\Cuda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda\envs\Cuda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Anaconda\envs\Cuda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Anaconda\envs\Cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
DF = pd.read_json('../Datasets/AIC Val/ds.jsonl', lines=True)
DF.head()
# DF = pd.read_csv('../Datasets\WikiHow\wikiHow.csv', nrows=2000)
# DF.columns = ['summary', 'paragraph']
# DF

,example_id,paragraph,summary
0,0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...


In [4]:
Paragraphs = DF['paragraph'].tolist()
Summaries = DF['summary'].tolist()

In [5]:
Train_Paragraphs, Test_Paragraphs, Train_Summaries, Test_Summaries = train_test_split(Paragraphs, Summaries, test_size=0.2, random_state=42)
Train_Paragraphs, Validation_Paragraphs, Train_Summaries, Validation_Summaries = train_test_split(Train_Paragraphs, Train_Summaries, test_size=0.2, random_state=42)

In [6]:
Model_Name = "facebook/mbart-large-50"
Tokenizer = MBartTokenizer.from_pretrained(Model_Name)
Model = MBartForConditionalGeneration.from_pretrained(Model_Name).to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
Paragraph_Max_Length = 512
Train_Paragraph_Encodings = Tokenizer(Train_Paragraphs, truncation=True, max_length=Paragraph_Max_Length, padding="max_length", return_tensors="pt").to(device)
Validation_Paragraph_Encodings = Tokenizer(Validation_Paragraphs, truncation=True, max_length=Paragraph_Max_Length, padding="max_length", return_tensors="pt").to(device)
Test_Paragraph_Encodings = Tokenizer(Test_Paragraphs, truncation=True, max_length=Paragraph_Max_Length, padding="max_length", return_tensors="pt").to(device)

Summary_Max_Length = 64
Train_Summary_Encodings = Tokenizer(Train_Summaries, truncation=True, max_length=Summary_Max_Length, padding="max_length", return_tensors="pt").to(device)
Validation_Summary_Encodings = Tokenizer(Validation_Summaries, truncation=True, max_length=Summary_Max_Length, padding="max_length", return_tensors="pt").to(device)
Test_Summary_Encodings = Tokenizer(Test_Summaries, truncation=True, max_length=Summary_Max_Length, padding="max_length", return_tensors="pt").to(device)

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, decoder_input_ids, decoder_attention_mask):
        self.input_ids = input_ids.to(device)
        self.attention_mask = attention_mask.to(device)
        self.decoder_input_ids = decoder_input_ids.to(device)
        self.decoder_attention_mask = decoder_attention_mask.to(device)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "decoder_input_ids": self.decoder_input_ids[idx],
            "decoder_attention_mask": self.decoder_attention_mask[idx]
        }

In [9]:
Train_Dataset = CustomDataset(
    Train_Paragraph_Encodings['input_ids'],
    Train_Paragraph_Encodings['attention_mask'],
    Train_Summary_Encodings['input_ids'],
    Train_Summary_Encodings['attention_mask']
)

Validation_Dataset = CustomDataset(
    Validation_Paragraph_Encodings['input_ids'],
    Validation_Paragraph_Encodings['attention_mask'],
    Validation_Summary_Encodings['input_ids'],
    Validation_Summary_Encodings['attention_mask']
)

Test_Dataset = CustomDataset(
    Test_Paragraph_Encodings['input_ids'],
    Test_Paragraph_Encodings['attention_mask'],
    Test_Summary_Encodings['input_ids'],
    Test_Summary_Encodings['attention_mask']
)

In [10]:
Train_Dataloader = DataLoader(Train_Dataset, batch_size=1)
Validation_Dataloader = DataLoader(Validation_Dataset, batch_size=1)
Test_Dataloader = DataLoader(Test_Dataset, batch_size=1)

In [11]:
Optimizer = torch.optim.AdamW(Model.parameters(), lr=1e-5)
Criterion = torch.nn.CrossEntropyLoss().to(device)

Epochs_Number = 5

Scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

for Epoch in range(Epochs_Number):
    Model.train()
    Total_Loss = 0

    Total_Rouge1 = 0.0
    Total_Rouge2 = 0.0
    Total_RougeL = 0.0
    Total_Batches = 0
    with tqdm(Train_Dataloader, desc=f"Epoch {Epoch + 1}/{Epochs_Number}",  unit="batch") as t:
        for Batch in t:
            Input_IDs = Batch["input_ids"].to(device)
            Attention_Mask = Batch["attention_mask"].to(device)
            Decoder_Input_IDs = Batch["decoder_input_ids"].to(device)
            Decoder_Attention_Mask = Batch["decoder_attention_mask"].to(device)

            # print("Input IDs Shape:", Input_IDs.shape)
            # print("Attention Mask Shape:", Attention_Mask.shape)
            # print("Decoder Input IDs Shape:", Decoder_Input_IDs.shape)
            # print("Decoder Attention Mask Shape:", Decoder_Attention_Mask.shape)

            Outputs = Model(
                input_ids=Input_IDs,
                attention_mask=Attention_Mask,
                decoder_input_ids=Decoder_Input_IDs,
                decoder_attention_mask=Decoder_Attention_Mask,
                labels=Decoder_Input_IDs
            )

            # IDs = Tokenizer.batch_decode(Input_IDs, skip_special_tokens=True)
            # decoded_output = Tokenizer.batch_decode(Outputs.logits.argmax(dim=-1), skip_special_tokens=True)
            # print(IDs)
            # print(decoded_output)

            Loss = Outputs.loss
            Total_Loss += Loss.item()

            Optimizer.zero_grad()
            Loss.backward()
            Optimizer.step()

            Reference_sentences = Tokenizer.batch_decode(Decoder_Input_IDs, skip_special_tokens=True)
            Generated_sentences = Tokenizer.batch_decode(Outputs.logits.argmax(dim=-1), skip_special_tokens=True)

            Scores = Scorer.score(''.join(Reference_sentences), ''.join(Generated_sentences))
            Total_Rouge1 += Scores['rouge1'].fmeasure
            Total_Rouge2 += Scores['rouge2'].fmeasure
            Total_RougeL += Scores['rougeL'].fmeasure
            Total_Batches += 1

            t.set_postfix(
                Loss_Average=Total_Loss / len(Train_Dataloader),
                loss=Loss.item(),
                Rouge1_Average=Total_Rouge1 / Total_Batches,
                Rouge2_Average=Total_Rouge2 / Total_Batches,
                RougeL_Average=Total_RougeL / Total_Batches
            )

Epoch 5/5: 100%|██████████| 98/98 [07:50<00:00,  4.80s/batch, Loss_Average=0.0138, Rouge1_Average=0.194, Rouge2_Average=0.0408, RougeL_Average=0.194, loss=0.00312] 


In [12]:
Model_Save_Path = 'mBart.pt'

In [13]:
torch.save(Model.state_dict(), Model_Save_Path)

In [ ]:
Model.eval()

Predicted_Summaries = []

with tqdm(Train_Dataloader, unit="batch") as t:
    for Batch in t:
        Input_IDs = Batch["input_ids"]
        Attention_Mask = Batch["attention_mask"]
        Outputs = Model.generate(
            input_ids=Input_IDs,
            attention_mask=Attention_Mask,
            max_length=Summary_Max_Length,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True
        )
         
        print(Outputs)
        IDs = Tokenizer.batch_decode(Input_IDs, skip_special_tokens=True)
        print(IDs)
        Batch_Summaries = Tokenizer.batch_decode(Outputs, skip_special_tokens=True)
        print(Batch_Summaries)
        Predicted_Summaries.extend(Batch_Summaries)


In [ ]:
Train_Paragraphs[0]

'التاريخ يقول ان غزو " ا آشوريا " لمصر حصل في القرن السابع قبل الميلاد واحتلت على اثره مصر لمدة تقارب المائة سنة تخللها مقاومة وحروب وقلاقل وثورات من المصريين لمقاومة الاحتلال الآشوري وفي النهاية استطاع المصريون طرد الآشوريين الغزاة لكن ما لبثوا ان غزاهم الفرس في منتصف القرن السادس قبل الميلاد وسقطت مصر في أيدي الفرس ورزحت تحت الاحتلال الفارسي قرابة 180 سنة بعد ان احتلها الآشوريون لمدة تقارب قرنا من الزمان ثم جاء اليونانيون بقيادة الإسكندر الأكبر المقدوني واحتل مصر وانهى الحقبة الفارسية في مصر واستمر حكم البطالمة لمصر قرابة ثلاثة قرون وما كادت مصر تخرج من نفوذ الحكم اليوناني حتى وقعت تحت الحكم الروماني واسمر حتى سقط على يد عمرو بن العاص فدخلت مصر تحت حكم العرب الى اليوم وان كان تخلله فترات حكم مصر خلالها المماليك والاخشيديون وأخيرا العثمانيون الأتراك وخضعت فترة قصيرة للانتداب البريطاني حتى نالت استقلالها وتأسست جمهورية مصر الحديثة واذا استثنينا فترة الانتداب البريطاني فان كل الحكومات من بعد الفتح الاسلامي لمصر حكومات إسلامية .. ومنذ الفتح الاسلامي لمصر توافد العرب زرافات الى مصر من جزي

In [ ]:
Model.load_state_dict(torch.load(Model_Save_Path, map_location=torch.device(device)))

<All keys matched successfully>

In [ ]:
print(Model)
print(type(Model))

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [ ]:
text = 'تاريخ أقول غزو " آشوريا " مصر حصل قرن سابع ميلاد احتال ثرا مصر مد تقارب المائة تخلل مقاوم حروب وقلاقل ثور مصري مقاوم احتلال الآشوري نهاية استطاع مصري طرد الآشوريين غزاة لبث غز فرس منتصف قرن سادس ميلاد سقط مصر يد فرس رزح احتلال فارس قراب ١٨٠ احتل الآشوريون مد تقارب قرن زم جاء يوناني قياد الإسكندر أكبر مقدوني حتل مصر وانهى حقب فارس مصر استمر حكم البطالمة مصر قراب قرون ما كاد مصر تخرج نفوذ حكم يونان وقع حكم رومان سمر سقط يد عمر العاص دخل مصر حكم عرب تخلل فتر حكم مصر خلال مماليك والاخشيديون أخير عثماني الأتراك خضع فترة قصير انتداب بريطاني نال استقلال تأسس جمهور مصر حديث واذا استثنى فترة انتداب بريطاني حكومة فتح الاسلامي مصر حكومة إسلام منذ فتح الاسلامي مصر توافد عرب زرافة مصر جزيرة عرب لاحظ تاريخ مصر سياسة حضار مدون بدأ تولي الاسرة فرعون ٣٥٠٠ ميلاد تعدي قدم حضار عالم تدوين حكام محكوم وقت افارقة محلي نقول عن اصحاب سلال جين e سلال جين لافارقة جنوب صحراء ( أمازيغ نوبي حبش ) انتهى حكم مصر خرج يد حكام محلي دخول الإسكندر أكبر مقدوني مصر ظن سلال جين j٢ دخل مصر قدوم الآشوريين فرس مصر دخل سلال جين j١ دخول الآشوريين لان الساميين اختلط الاشوريين الامبراطورية الآشورية كانو مكون اجتماع إمبراطوري الاشورية بابلي سومر عراق والشام ازداد ضخ سلال جين j١ مصر فتح الاسلامي مصر استنتج سلال جين مصري أصلي e سلال جين j١ j٢ وافد أجانب مصر ذلك جبى نسى سلال جين الهندواوروبية r دخل مصر دخول البطالمة يونان مصر سلال جين أوروبي الاخرى دخل مصر دخول روم مصر بناء تقدم مستغرب ظهر المومياوات صلى عمر قارب ألف ميلاد سلال جين ( j١ j٢ r ) جوار سلال جين محل e'

input_ids = Tokenizer.encode(text, return_tensors="pt").to(device)
summary_ids = Model.generate(input_ids, max_length=100, num_beams=4, decoder_start_token_id=Model.config.pad_token_id)

summary = Tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)


In [ ]:
# print(summary_ids)
# print(input_ids)
print(len(text))
print(text)
print(len(summary))
print(summary)

1436
تاريخ أقول غزو " آشوريا " مصر حصل قرن سابع ميلاد احتال ثرا مصر مد تقارب المائة تخلل مقاوم حروب وقلاقل ثور مصري مقاوم احتلال الآشوري نهاية استطاع مصري طرد الآشوريين غزاة لبث غز فرس منتصف قرن سادس ميلاد سقط مصر يد فرس رزح احتلال فارس قراب ١٨٠ احتل الآشوريون مد تقارب قرن زم جاء يوناني قياد الإسكندر أكبر مقدوني حتل مصر وانهى حقب فارس مصر استمر حكم البطالمة مصر قراب قرون ما كاد مصر تخرج نفوذ حكم يونان وقع حكم رومان سمر سقط يد عمر العاص دخل مصر حكم عرب تخلل فتر حكم مصر خلال مماليك والاخشيديون أخير عثماني الأتراك خضع فترة قصير انتداب بريطاني نال استقلال تأسس جمهور مصر حديث واذا استثنى فترة انتداب بريطاني حكومة فتح الاسلامي مصر حكومة إسلام منذ فتح الاسلامي مصر توافد عرب زرافة مصر جزيرة عرب لاحظ تاريخ مصر سياسة حضار مدون بدأ تولي الاسرة فرعون ٣٥٠٠ ميلاد تعدي قدم حضار عالم تدوين حكام محكوم وقت افارقة محلي نقول عن اصحاب سلال جين e سلال جين لافارقة جنوب صحراء ( أمازيغ نوبي حبش ) انتهى حكم مصر خرج يد حكام محلي دخول الإسكندر أكبر مقدوني مصر ظن سلال جين j٢ دخل مصر قدوم الآشوريين فرس مصر دخل سلال

In [ ]:
Train_Paragraphs[0]

'التاريخ يقول ان غزو " ا آشوريا " لمصر حصل في القرن السابع قبل الميلاد واحتلت على اثره مصر لمدة تقارب المائة سنة تخللها مقاومة وحروب وقلاقل وثورات من المصريين لمقاومة الاحتلال الآشوري وفي النهاية استطاع المصريون طرد الآشوريين الغزاة لكن ما لبثوا ان غزاهم الفرس في منتصف القرن السادس قبل الميلاد وسقطت مصر في أيدي الفرس ورزحت تحت الاحتلال الفارسي قرابة 180 سنة بعد ان احتلها الآشوريون لمدة تقارب قرنا من الزمان ثم جاء اليونانيون بقيادة الإسكندر الأكبر المقدوني واحتل مصر وانهى الحقبة الفارسية في مصر واستمر حكم البطالمة لمصر قرابة ثلاثة قرون وما كادت مصر تخرج من نفوذ الحكم اليوناني حتى وقعت تحت الحكم الروماني واسمر حتى سقط على يد عمرو بن العاص فدخلت مصر تحت حكم العرب الى اليوم وان كان تخلله فترات حكم مصر خلالها المماليك والاخشيديون وأخيرا العثمانيون الأتراك وخضعت فترة قصيرة للانتداب البريطاني حتى نالت استقلالها وتأسست جمهورية مصر الحديثة واذا استثنينا فترة الانتداب البريطاني فان كل الحكومات من بعد الفتح الاسلامي لمصر حكومات إسلامية .. ومنذ الفتح الاسلامي لمصر توافد العرب زرافات الى مصر من جزي

In [ ]:
# Scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Scores = []

# for predicted, test in zip(Predicted_Summaries, Test_Summaries):
#     score = Scorer.score(predicted, test)
#     Scores.append(score)

# Metrics = {
#     "rouge-1": [score["rouge1"].fmeasure for score in Scores],
#     "rouge-2": [score["rouge2"].fmeasure for score in Scores],
#     "rouge-l": [score["rougeL"].fmeasure for score in Scores]
# }

# print(Metrics)

In [ ]:
Scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
Scores = Scorer.score(Predicted_Summaries[0], Test_Summaries[0])

Metrics = {
    "rouge-1": Scores["rouge1"].fmeasure,
    "rouge-2": Scores["rouge2"].fmeasure,
    "rouge-l": Scores["rougeL"].fmeasure,
}

print(Metrics)

{'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0}


In [ ]:
Test_Paragraphs[0]

'اعلنت " لجنة جائزة بشارة الخوري للتوعية الديموقراطية " انها انجزت المسابقة التي نظمتها حول " دور الرئيس بشارة الخوري في تحقيق الميثاق والاستقلال " بالتعاون مع وزارة التربية والتعليم العالي وبدعم من " مؤسسة فريديريتش ايبرت "، والتي شارك فيها طلاب الصف الثانوي الثاني بفرعيه العلمي والادبي والذين ينتمون الى مدارس رسمية وخاصة في المحافظات اللبنانية ، لافتة الى ان اعلان النتائج وتوزيع الجوائز سيتم خلال احتفال يقام عند الخامسة من بعد ظهر الخميس 30 الحالي في قصر الاونيسكو . واوضحت اللجنة في بيان " ان المسابقة تناولت تاريخ لبنان الحديث من قيام دولة لبنان الكبير في الاول من ايلول 1920، الى الجلاء في 31 كانون الاول 1946، دور الرئيس بشارة الخوري في تحقيق الوحدة الوطنية واهمية الالتزام بالميثاق الوطني للمحافظة على الاستقلال ". واشارت " الى ان الطلاب المشاركين اخضعوا في مرحلة اولى لامتحان خطي فاز في نهايته 69 طالبا ، ثم اشترك 17 منهم ممن نالوا العلامات الاعلى في المرحلة الثانية من المسابقة التي تضمنت امتحانا شفهيا امام لجنة فاحصة من الاساتذة المختصين اجابوا فيه على اسئلة تناولت فترة ولادة الاستقلا

In [ ]:
Predicted_Summaries[0]

''

In [ ]:
Test_Summaries[0]

'قالت لجنة جائزة بشارة الخوري أنها اجتازت المسابقة التي نظمتها حول بشارة الخوري في أن يحقق الاستقلال، وقام طلبة الصف الثاني الثانوي بالمشاركة في مؤسسة فريديريتش ايبريت، وأن توزيع الجوائز يتم في احتفال يحدث الخامسة بعد الظهر، واللجنة قالت في بيان أن المسابقة كانت تحتوي على تاريخ لبنان الحديث، وأن بشارة الخوري حافظ على الالتزام بالميثاق لكي ينال الاستقلال، وأعلن عن أسماء الطلبة الفائزين في احتفال الأونيسكو.'